In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
np.random.seed(42)
tf.random.set_seed(42)

### Carregando o conjunto de dados para regressão. 

+ Vamos usar o conjunto de dados habitacional da Califórnia e criar um regressor com uma rede neural.

+ Depois de carregar os dados, dividimos em um conjunto de treinamento, um conjunto de validação e um conjunto de teste, e padronizamos todos os atributos.

In [3]:
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

### Criando e compilando o modelo.

+ Usamos a API sequencial.

In [4]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

### Treinando o modelo.

In [5]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
363/363 [==============================] - 2s 4ms/step - loss: 1.8866 - val_loss: 0.7126
Epoch 2/10
363/363 [==============================] - 1s 3ms/step - loss: 0.6577 - val_loss: 0.6880
Epoch 3/10
363/363 [==============================] - 1s 3ms/step - loss: 0.5934 - val_loss: 0.5803
Epoch 4/10
363/363 [==============================] - 2s 4ms/step - loss: 0.5557 - val_loss: 0.5166
Epoch 5/10
363/363 [==============================] - 2s 5ms/step - loss: 0.5272 - val_loss: 0.4895
Epoch 6/10
363/363 [==============================] - 3s 7ms/step - loss: 0.5033 - val_loss: 0.4951
Epoch 7/10
363/363 [==============================] - 2s 4ms/step - loss: 0.4854 - val_loss: 0.4861
Epoch 8/10
363/363 [==============================] - 1s 4ms/step - loss: 0.4709 - val_loss: 0.4554
Epoch 9/10
363/363 [==============================] - 2s 4ms/step - loss: 0.4578 - val_loss: 0.4413
Epoch 10/10
363/363 [==============================] - 1s 4ms/step - loss: 0.4474 - val_loss: 0.4379

### Salvando o modelo.

In [6]:
model.save("my_keras_model.h5")

### Carregando o modelo salvo.

In [7]:
loaded_model = keras.models.load_model("my_keras_model.h5")

### Realizando predições com o modelo salvo.

In [8]:
X_new = X_test[:3]

loaded_model.predict(X_new)

1/1 [==============================] - 0s 162ms/step


array([[0.54002357],
       [1.650597  ],
       [3.0098243 ]], dtype=float32)

### Salvando os pesos de modelos criados a partir de subclasses da classe `Model`.

+ O método `save()` e a função `load_model` funcionam ao usar as APIs sequencial e funcional, mas, infelizmente, não ao usar subclasses da classe `Model`. 

+ Porém, podemos usar os métodos `save_weights()` e `load_weights()` para pelo menos salvar e restaurar os parâmetros do modelo. 

+ Entretanto, precisaremos salvar e restaurar todo o resto nós mesmos.

In [9]:
model.save_weights("my_keras_weights.ckpt")

#### Importante

Para carregar os pesos, precisamos antes ter um modelo instanciado e compilado.

In [10]:
model.load_weights("my_keras_weights.ckpt")

Depois dos pesos terem sido carregados, podemos fazer predições normalmente.

In [11]:
model.predict(X_new)

1/1 [==============================] - 0s 118ms/step


array([[0.54002357],
       [1.650597  ],
       [3.0098243 ]], dtype=float32)